In [2]:
import os
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import numpy as np


In [3]:
sp500=yf.download('^GSPC', start='2020-06-01', interval='1mo',auto_adjust=True)
msft=yf.download('KO', start='2020-06-01', interval='1mo',auto_adjust=True)

sp500=sp500[['Close']].reset_index()
msft=msft[['Close']].reset_index()

#Renombrar columnas
sp500.columns=['Date','SP500']
msft.columns=['Date','MSFT']

merged=pd.merge(sp500, msft, on='Date',how='inner')

#Calcular los retornos
merged['SP500_ret'] = merged['SP500'].pct_change()
merged['MSFT_ret'] = merged['MSFT'].pct_change() 

df_reg=merged.iloc[1:-1].copy()
display(df_reg.head())
y=df_reg['MSFT_ret']
X=sm.add_constant(df_reg['SP500_ret'])
reg_msft=sm.OLS(y,X).fit()
print(reg_msft.summary())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,SP500,MSFT,SP500_ret,MSFT_ret
1,2020-07-01,3271.120117,40.592743,0.055101,0.066902
2,2020-08-01,3500.310059,42.560497,0.070065,0.048476
3,2020-09-01,3363.000000,42.423027,-0.039228,-0.003230
4,2020-10-01,3269.959961,41.631653,-0.027666,-0.018654
5,2020-11-01,3621.629883,44.698151,0.107546,0.073658


                            OLS Regression Results                            
Dep. Variable:               MSFT_ret   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     12.19
Date:                Sat, 02 Aug 2025   Prob (F-statistic):           0.000918
Time:                        12:14:51   Log-Likelihood:                 101.63
No. Observations:                  61   AIC:                            -199.3
Df Residuals:                      59   BIC:                            -195.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0050      0.006      0.810      0.4

In [31]:

#Calcular beta de una acción respecto al S&P 500
def calcular_beta(ticker):

    sp500 = yf.download('^GSPC', start='2020-06-01',interval='1mo',auto_adjust=True)

    # Descargar datos de la acción
    accion = yf.download(ticker, start='2020-06-01',interval='1mo',auto_adjust=True)
    
    # Calcular retornos diarios
    sp500['Retorno_SPY'] = sp500['Close'].pct_change()
    accion['Retorno_MF'] = accion['Close'].pct_change()
    
    # Eliminar filas con valores NaN
    data = pd.concat([sp500['Retorno_SPY'], accion['Retorno_MF']], axis=1).dropna()
    display(data.head())
    # Ajustar el modelo de regresión lineal
    modelo = sm.OLS(data['Retorno_MF'], sm.add_constant(data['Retorno_SPY'])).fit()
    
    return modelo.params[1]  # Retorna la beta

In [32]:
print(calcular_beta('MSFT'))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Retorno_SPY,Retorno_MF
Date,,
2020-07-01,0.055101,0.007371
2020-08-01,0.070065,0.100093
2020-09-01,-0.039228,-0.065142
2020-10-01,-0.027666,-0.037370
2020-11-01,0.107546,0.057293


1.0295425827397306


/var/folders/jr/t_k402pj3bg57t7p56rgmknr0000gn/T/ipykernel_2448/3447652064.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return modelo.params[1]  # Retorna la beta


### Laboratorio: 
#### Aca generarmos las series de tiemo de ejemplo para ver que pinta tiene

In [9]:
rho=0.7
sigma=1
df=pd.DataFrame(np.random.normal(loc=0, scale=1, size=100),columns=['ruido_blanco'])

## Ahora aplicamos el test de df a ver que me dice. 

from statsmodels.tsa.stattools import adfuller 
def test_dickey_fuller(series):
    """Realiza la prueba de Dickey-Fuller aumentado."""
    result = adfuller(series, autolag='AIC')
    print(f'Estadístico ADF: {result[0]}')
    print(f'Valor p: {result[1]}')
    print('Valores críticos:')
    for key, value in result[4].items():
        print(f'  {key}: {value}')

test_dickey_fuller(df['ruido_blanco'])

## El p-value es significativo por o tanto la serie es Estacionaria???

### Hipótesis nula H0: la serie tiene una raíz unitaria (no estacionaria).
### Hipógtesis alternativa H1: la serie es estacionaria
## como el p-value es significativo, dado que el estaditico ADF esta mas a la izquierda que cualquier valor crítico RH0 y por lo tanto es estacionaria

Estadístico ADF: -9.520077013471598
Valor p: 3.0816939063257074e-16
Valores críticos:
  1%: -3.498198082189098
  5%: -2.891208211860468
  10%: -2.5825959973472097
